In [ ]:
from helpers import *

In [ ]:
base_path = '..'

In [ ]:

taz2edge_xml = 'taz_new.xml'
net_xml = 'SFO.net.xml'
fixed_routes_xml = f'{base_path}/5hr_route_choice_set.csv'
od_duration_seconds = 5*60 

# duration of sample time for simulation output statistics
simulation_stat_freq_sec = od_duration_seconds
sim_end_time = od_duration_seconds
additional_xml = f'additional.add_statfreq{od_duration_seconds}.xml'

# suffix of simulation output edge file
EDGE_OUT_STR = 'edge_data_SFO.xml'
TRIPS2ODS_OUT_STR = 'trips.xml'
SUMO_PATH = '/usr/local/opt/sumo/share/sumo'

In [ ]:
gt_version_str = 'v4'


# # gt v5:
# mean_od_val = 100
# num_ods = 100

# gt v4:
mean_od_val = 100
num_ods = 10

# # gt v3:
# mean_od_val = 0.05

# # gt v2:
# mean_od_val = 0.5
#num_ods = routes_df.shape[0]


# # gt v1:
# mean_od_val = 100
#num_ods = routes_df.shape[0]


print('if you want to optimize them all (~86k) set num_ods as defined in commented line below')
#num_ods = routes_df.shape[0]

In [ ]:
od_xml = f'gt_od_{gt_version_str}.xml'
file_gt = f'{base_path}/gt_od_{gt_version_str}.xml'
file_gt_edges = f'{base_path}/gt_edges_{gt_version_str}.csv'
prefix_output = f'gt_{gt_version_str}'

In [ ]:
# Get GT OD
print("Reading:",file_gt)
tree = ET.parse(file_gt)
root = tree.getroot()
gt_od_df =  xml2df_str(root, 'tazRelation')

gt_od_df.head()

In [ ]:
print("Reading:",fixed_routes_xml)
routes_df = pd.read_csv(fixed_routes_xml, index_col=0)

In [ ]:
od_xml

In [ ]:
prefix_output

In [ ]:
# simulate gt od
simulate_od(od_xml, 
            prefix_output, 
            base_path, 
            net_xml, 
            taz2edge_xml, 
            additional_xml,
            routes_df,
            sim_end_time,
            TRIPS2ODS_OUT_STR)

In [ ]:
def od_xml_to_df(file_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    gt_od_df =  xml2df_str(root, 'tazRelation')
    
    gt_od_vals = gt_od_df['count'].astype(float)
    print('total GT demand: ',gt_od_vals.sum())

    return gt_od_df

In [ ]:
gt_od_df = od_xml_to_df(file_gt)

## Vary OD values adding Gaussian noise

In [ ]:
gt_od_df['count'].astype(float).to_numpy()

In [ ]:
num_epsilon_iter = 10
ods_epsilon = []

# Base OD which we will update their count entries
base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()

for o1 in range(num_epsilon_iter):
    print(f"########### Epsilon {o1} ###########")

    file_od_epsilon_xml = f'gt_od_{gt_version_str}_epsilon{o1}.xml'
    prefix_output = f'epsilon{o1}'

    # Generate OD
    curr_od = gt_od_vals.copy()
    curr_od = np.random.normal(gt_od_vals, gt_od_vals/30)

    print(f'total expected GT demand: {np.sum(curr_od)}')

    ###
    # create OD xml file 
    ###
    base_od['count'] = curr_od
    # round to 1 decimal point
    base_od['count'] = [round(elem, 1) for elem in base_od['count']]     
    base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
    create_taz_xml(file_od_epsilon_xml, base_od, od_duration_seconds, base_path)
    ods_epsilon.append(curr_od)
    
    # simulate gt od
    simulate_od(file_od_epsilon_xml, 
                prefix_output, 
                base_path, 
                net_xml, 
                taz2edge_xml, 
                additional_xml, 
                routes_df,
                sim_end_time,
                TRIPS2ODS_OUT_STR)

In [ ]:
num_gt_edges = 10
sim_edge_out = f'{base_path}/{prefix_output}_{EDGE_OUT_STR}'
df_edge_gt, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out, prefix_output,SUMO_PATH)

df_edge_gt.head()

In [ ]:
gt_edge_data = df_edge_gt\
        .sort_values(by=['interval_nVehContrib'], ascending=False)\
        .iloc[:num_gt_edges]

In [ ]:
edge_epsilon_all = []
num_epsilon_iter = 10
for o1 in range(num_epsilon_iter):
    curr_prefix = f'epsilon{o1}'
    sim_edge_out = f'{base_path}/{curr_prefix}_{EDGE_OUT_STR}'
    curr_loop_stats, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out, curr_prefix,SUMO_PATH)
    edge_epsilon_all.append(curr_loop_stats)

In [ ]:
loss_all = []
for o1 in range(num_epsilon_iter):
    curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, edge_epsilon_all[o1])
    print(curr_loss)
    loss_all.append(curr_loss)


In [ ]:
print("Number of ODs:",num_ods)
gt_od_df = routes_df[['fromTaz', 'toTaz']].drop_duplicates().iloc[:num_ods]